In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import re

In [2]:
data=pd.read_csv("SpamCollectionSMS.txt",sep="\t",names=["label","text"])

In [3]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [5]:
data.isnull().any()

label    False
text     False
dtype: bool

In [6]:
def remove_punc(text):
    text="".join([char for char in text if char not in string.punctuation])
    return text
data.text=data.text.apply(remove_punc)

In [7]:
def tokenize(text):
    tokens=re.split("\W+",text.lower())
    return tokens
data.text=data.text.apply(tokenize)

In [8]:
def remove_stopwords(text):
    token=[word for word in text if word not in stopwords.words("english")]
    return token
data.text=data.text.apply(remove_stopwords)

In [11]:
wn=x=nltk.WordNetLemmatizer()
def lematizing(text):
    token="".join([wn.lemmatize(word) for word in text])
    return token
data.text=data.text.apply(lematizing)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv1=CountVectorizer()
x_count=cv1.fit_transform(data.text)
print(x_count.shape)
print(cv1.get_feature_names())

ImportError: cannot import name 'CountVectorizer' from 'sklearn.feature_extraction' (C:\Users\nipun\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\__init__.py)